# Primera entrega

### Instalación de requerimietos adicionales

In [4]:
!pip3 install -U -q ipywidgets
!jupyter labextension install @jupyter-widgets/jupyterlab-manager #(Installing into JupyterLab 1 or 2)

### Importación de librerías

In [1]:
from ipywidgets import interact, interactive
import ipywidgets as widgets

import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
from skimage import io, color
from ipywidgets import Layout
import os
%matplotlib inline


## Análisis de los espacios de color y operaciones sobre el histograma

In [2]:
import sys
sys.path.append('./utils/')
from utils.funciones_utiles import *

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()
out6 = widgets.Output()
out7 = widgets.Output()

def analisis_espacio_color(color_space, channel, img, a, b, c, f, lineal):
    
    
    img = cv2.imread(os.path.join("img/",img+'.jpg'))
    
    img_tittle = color_space
    channel_tittle = color_space[channel-1]
    
    cs = color_space
    color_space = cv2.COLOR_BGR2RGB if color_space=='RGB' else \
                  cv2.COLOR_BGR2YUV if color_space=='YUB' else \
                  cv2.COLOR_BGR2HSV if color_space=='HSV' else \
                  cv2.COLOR_BGR2LAB if color_space=='HSL' else None
    
    img_space  = cv2.cvtColor(img, color_space)
    
    if not color_space:
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_R = img_rgb[:,:,0]
        img_G = img_rgb[:,:,1]
        img_B = img_rgb[:,:,2]
        
        if cs=='CMY':
            
            img_space[:,:,0] = 255 - img_R
            img_space[:,:,1] = 255 - img_G
            img_space[:,:,2] = 255 - img_B
            
        elif cs=='YIQ':
            
            img_space[:,:,0] = 0.299*img_R + 0.587*img_G + 0.114*img_B
            img_space[:,:,1] = 0.596*img_R - 0.274*img_G - 0.322*img_B
            img_space[:,:,2] = 0.211*img_R - 0.523*img_G + 0.312*img_B
        
        elif cs=='HSL':
            
            img_space = rgb2hsl(img_rgb)
            
    if lineal == 'True':
        img_space = apply_linear_function(img_space, f, [a,b,c])
        img_channel = img_space[:,:,channel-1]
    else:
        img_space = apply_non_linear_function(img_space, f, [a,b])
        img_channel = img_space[:,:,channel-1]
    
    fig, arreglo_plots = plt.subplots(1,2, figsize=(15, 8))
    colors = ('r','g','b')    
    
    arreglo_plots[0].set_title(img_tittle)
    arreglo_plots[0].imshow(img_channel, cmap="gray")
    arreglo_plots[1].set_title('Histograma Canal '+ channel_tittle)
    img_array = img_channel.ravel()
    arreglo_plots[1].hist(img_array,histtype='step', bins=255, 
                            range=(0.0, 255.0),density=True, color=colors[channel-1],
                             )
    plt.show()

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width= 'max-content',)

def show_interact(function, max_v=0, min_v=0, step=0, lineal=True):
    interact(
        analisis_espacio_color,
        color_space=widgets.ToggleButtons(
                                        options=['RGB','CMY','YIQ','YUB','HSL','HSV','LAB'],
                                        description='Escoge el espacio de color a usar:',
                                        disabled=False,
                                        button_style='info',
                                        tooltips=['modelo aditivo', 'modelo sustractivo', 'Description of fast',
                                                  'separa la información de intensidad o luminancia',
                                                  'ancho de banda reducido para los componentes de crominancia',
                                                  'fácilmente interpretables y legibles por un humano ...',
                                                  '... métodos en los que la representación del componente de color no es lineal',
                                                  'L -> información sobre luminosidad, a* y b* -> información de color'],
                                        layout=box_layout,
                                        style= {'description_width': '200px'}
                                        ),
        channel=widgets.RadioButtons(description="Escoge el canal a visualizar:", options=[1, 2, 3],style= {'description_width': 'auto'}, ),

        a = widgets.FloatSlider(value=0,
                                    min=min_v,
                                    max=max_v,
                                    step=step,
                                    description='canal 1:',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.1f',
                                    layout=Layout(visibility='hidden') if (not lineal and function!='T. gamma') else Layout()
                                ),
        b = widgets.FloatSlider(value=0,
                                    min=min_v,
                                    max=max_v,
                                    step=step,
                                    description='canal 2:',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.1f',
                                    layout=Layout(visibility='hidden') if (not lineal and function!='T. gamma') else Layout()
                                ),
        c = widgets.FloatSlider(value=0,
                                    min=min_v,
                                    max=max_v,
                                    step=step,
                                    description='canal 3:',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.1f',
                                    layout=Layout(visibility='hidden') if not lineal else Layout()
                                ),
        f = widgets.Text(
                                value=function,
                                placeholder='Type something',
                                description='String:',
                                disabled=False,
                                layout=Layout(visibility='hidden')
                            ),
        lineal = widgets.Text(
                                value=str(lineal),
                                placeholder='Type something',
                                description='String:',
                                disabled=False,
                                layout=Layout(visibility='hidden')
                            ),
        
        img=widgets.Dropdown(
            options=["B1", "B2", "B3","B4", "B5", "B6","B7", "B8", "B9","B10", "B11", "B12"], value="B1", description="Escoger imagen:",
            style= {'description_width': 'auto'}
        ),
    );
    
    
with out1:
    show_interact('suma', 255, 0, 1)
    
with out2:
    show_interact('resta', 255, 0,  1)
    
with out3:
    show_interact('multiplicacion', 5, 0, 1)
    
with out4:
    show_interact('division', 1, 0, 0.01)
    
with out5:
    show_interact('T. gamma',2,0,0.1, lineal=False)

with out6:
    show_interact('Ec. histograma', lineal=False)

with out7:
    show_interact('Exp. histograma', lineal=False)
    
tab = widgets.Tab(children = [out1, out2, out3, out4, out5, out6, out7])
tab.set_title(0, 'Suma')
tab.set_title(1, 'Resta')
tab.set_title(2, 'Multiplicación')
tab.set_title(3, 'División')
tab.set_title(4, 'T. gamma')
tab.set_title(5, 'Ec. histograma')
tab.set_title(6, 'Exp. histograma')

display(tab)

# Operaciones sobre el histograma

## Transformaciones lineales